<a href="https://colab.research.google.com/github/IshanThapa/NLP_BoilerPlates/blob/master/Interactive_BERT_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [2]:
!pip install transformers torch scikit-learn --quiet

In [3]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import json
import os
import time

In [4]:
print("Loading BERT model (this may take ~30-90 seconds on first run)...")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
print("BERT loaded. Device:", device)

Loading BERT model (this may take ~30-90 seconds on first run)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERT loaded. Device: cpu


In [6]:
def get_embedding(text: str):

  inputs = tokenizer(
  text,
  return_tensors="pt",
  truncation=True,
  padding=True,
  max_length=128
  )
  #move inputs to the model device

  inputs = {k: v.to(device) for k, v in inputs.items()}
  with torch.no_grad():
    outputs = model(**inputs)

  # outputs.last_hidden_state shape: (batch_size=1, seq_len, hidden_dim)
  emb = outputs.last_hidden_state.mean(dim=1).cpu().numpy()[0]
  return emb

In [7]:
QA_FILE = "qa_data.json"

def load_qa(path=QA_FILE):
  """Load Q&A from JSON; create initial file if not found."""
  if os.path.exists(path):
    with open(path, "r", encoding="utf-8") as f:
      return json.load(f)
  else:
    #if not exists, create with INITIAL_QA (defined below)
    with open(path, "w", encoding="utf-8") as f:
      json.dump(INITIAL_QA, f, ensure_ascii=False, indent=2)
    return INITIAL_QA.copy()
def save_qa(qa_dict, path=QA_FILE):
  """Save Q&A dictionary to disk."""
  with open(path, "w", encoding="utf-8") as f:
    json.dump(qa_dict, f, ensure_ascii=False, indent=2)

In [8]:
INITIAL_QA = {
"Hi": "Hello! Welcome to Suds & Shine Laundry ￿. How can I help you today?",
"Hello": "Hi there! Welcome to Suds & Shine Laundry. How may I assist?",
"What services do you offer?": "We offer wash & fold, dry cleaning,␣
↪ironing, stain removal, sofa and curtain cleaning, and pickup & delivery.",
"What are your working hours?": "We're open Monday–Saturday, 8:00 AM to 8:
↪00 PM. Closed on Sundays.",
"Where are you located?": "We're at 123 Clean Street, Near Karve Nagar,␣
↪Pune.",
"How much does a basic wash cost?": "A basic wash & fold starts at ￿50 per␣
↪kg.",
"How much for dry cleaning a shirt?": "Dry cleaning for a shirt is ￿80.",
"Do you do same-day service?": "Yes — for small loads, we offer same-day␣
↪service if dropped off before 10:00 AM (subject to availability).",
"Do you offer pickup and delivery?": "Yes! Free pickup & delivery within a␣
↪5 km radius. Charges apply beyond that.",
"How long does it take?": "Turnaround is usually 24–48 hours for regular␣
↪service; express options are available.",
"Can you remove tough stains?": "We handle many tough stains. Please point␣
↪them out during drop-off for the best treatment.",
"Do you wash delicate fabrics?": "Yes — we have special care processes for␣
↪silk, wool, and other delicates.",
"Do you accept online payments?": "We accept cash, UPI, and card payments␣
↪on delivery.",
"Do you have membership plans?": "Yes — monthly packages start at ￿999 and␣
↪include discounts on every order.",
"How do I track my order?": "You can track orders via SMS/WhatsApp updates.␣
↪Share your order ID with us.",
"What is your contact number?": "Call or WhatsApp us at +91 98765 43210.",
"Do you provide garment repairs?": "Minor repairs like small stitches and␣
↪buttons are available at an extra charge.",
"Are your detergents eco-friendly?": "We offer eco-friendly cleaning on␣
↪request (may be an extra charge).",
"Can I schedule a regular pickup?": "Yes — weekly or bi-weekly pickups can␣
↪be scheduled.",
"Do you offer wedding dress cleaning?": "Yes — we offer specialist cleaning␣
↪for wedding gowns (appointment required).",
"What is your cancellation policy?": "You can cancel within 1 hour of␣
↪booking without charge; after that, a small fee applies."
}

SyntaxError: unterminated string literal (detected at line 4) (ipython-input-769172272.py, line 4)